In [2]:
import numpy as np
import pandas as pd

import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

from config import fetch_data

## **TRANSACTION MONITORING**

### Realtime Transaction Gauge

In [3]:
realtime_transaction_gauge_query = "SELECT * FROM gold.analytics_realtime_transaction_monitoring"
realtime_transaction_gauge_df = fetch_data(realtime_transaction_gauge_query)

tpm = realtime_transaction_gauge_df['transactions_per_minute_5min'].iloc[0]
tpm_15 = realtime_transaction_gauge_df['transactions_per_minute_15min'].iloc[0]

fig = go.Figure(go.Indicator(
    mode="gauge+number+delta",
    value=tpm,
    delta={'reference': tpm_15, 'valueformat': '.2f'},
    title={
        'text': "Transactions per Minute<br><sub>(Last 5 min vs 15 min avg)</sub>"},
    number={'suffix': " /min"},
    gauge={
        'axis': {'range': [None, tpm * 2.5]},
        'bar': {'color': "darkblue"},
        'steps': [
            {'range': [0, tpm * 0.7], 'color': "lightgray"},
            {'range': [tpm * 0.7, tpm * 1.3], 'color': "lightgreen"},
            {'range': [tpm * 1.3, tpm * 2.5], 'color': "yellow"}
        ],
        'threshold': {
            'line': {'color': "red", 'width': 4},
            'thickness': 0.75,
            'value': tpm * 2
        }
    }
))

fig.update_layout(
    height=400, title_text="Real-time Transaction Throughput")

In [4]:
realtime_transaction_gauge_df

,snapshot_time,current_hour,transactions_current_hour,volume_current_hour,transactions_last_15min,volume_last_15min,transactions_last_5min,transactions_per_minute_15min,transactions_per_minute_5min,fraud_alerts_15min,...,avg_transactions_per_customer,transactions_same_hour_yesterday,change_vs_yesterday_pct,system_status,mobile_15min,online_15min,atm_15min,branch_15min,mobile_pct,online_pct
0,2025-12-27 09:08:31.690628+00:00,2025-12-27 09:00:00+00:00,0,None,0,None,0,0.0,0.0,0,...,None,0,None,NORMAL,0,0,0,0,None,None


In [6]:
realtime_metrics_dashboard_query = "SELECT * FROM gold.analytics_realtime_transaction_monitoring"
realtime_metrics_dashboard_df = fetch_data(realtime_metrics_dashboard_query)

fig = make_subplots(
    rows=3, cols=3,
    specs=[[{"type": "indicator"}] * 3] * 3,
    subplot_titles=(
        "Transactions (Hour)", "Transactions (15min)", "Transactions (5min)",
        "Fraud Alerts", "Decline Rate", "Avg Amount",
        "Unique Customers", "Mobile %", "Status"
    )
)

metrics = [
    (realtime_metrics_dashboard_df['transactions_current_hour'].iloc[0], ""),
    (realtime_metrics_dashboard_df['transactions_last_15min'].iloc[0], ""),
    (realtime_metrics_dashboard_df['transactions_last_5min'].iloc[0], ""),
    (realtime_metrics_dashboard_df['fraud_alerts_15min'].iloc[0], ""),
    (realtime_metrics_dashboard_df['decline_rate_pct_15min'].iloc[0], "%"),
    (realtime_metrics_dashboard_df['avg_amount_current_hour'].iloc[0], "$"),
    (realtime_metrics_dashboard_df['unique_customers_current_hour'].iloc[0], ""),
    (realtime_metrics_dashboard_df['mobile_pct'].iloc[0], "%"),
    (1 if realtime_metrics_dashboard_df['system_status'].iloc[0] == 'NORMAL' else 0, "")
]

positions = [(1, 1), (1, 2), (1, 3), (2, 1), (2, 2),
                (2, 3), (3, 1), (3, 2), (3, 3)]
colors = ['#636EFA', '#636EFA', '#636EFA', '#EF553B', '#FFA15A',
            '#00CC96', '#AB63FA', '#00CC96',
            '#00CC96' if metrics[8][0] == 1 else '#EF553B']

for (value, suffix), (row, col), color in zip(metrics, positions, colors):
    fig.add_trace(
        go.Indicator(
            mode="number",
            value=value,
            number={'suffix': suffix, 'valueformat': ",.0f" if suffix in [
                "", "$"] else ",.1f"},
            domain={'x': [0, 1], 'y': [0, 1]},
            title={'font': {'size': 12}}
        ),
        row=row, col=col
    )

fig.update_layout(
    title_text=f"Real-time Transaction Dashboard ({realtime_metrics_dashboard_df['snapshot_time'].iloc[0]})",
    height=700,
    showlegend=False
)
fig.show()

In [7]:
realtime_metrics_dashboard_df

,snapshot_time,current_hour,transactions_current_hour,volume_current_hour,transactions_last_15min,volume_last_15min,transactions_last_5min,transactions_per_minute_15min,transactions_per_minute_5min,fraud_alerts_15min,...,avg_transactions_per_customer,transactions_same_hour_yesterday,change_vs_yesterday_pct,system_status,mobile_15min,online_15min,atm_15min,branch_15min,mobile_pct,online_pct
0,2025-12-27 09:20:18.200708+00:00,2025-12-27 09:00:00+00:00,0,None,0,None,0,0.0,0.0,0,...,None,0,None,NORMAL,0,0,0,0,None,None


In [8]:
realtime_channel_distribution_query = "SELECT * FROM gold.analytics_realtime_transaction_monitoring"
realtime_channel_distribution_df = fetch_data(realtime_channel_distribution_query)

channels = ['Mobile', 'Online', 'ATM', 'Branch']
values = [
    realtime_channel_distribution_df['mobile_15min'].iloc[0],
    realtime_channel_distribution_df['online_15min'].iloc[0],
    realtime_channel_distribution_df['atm_15min'].iloc[0],
    realtime_channel_distribution_df['branch_15min'].iloc[0]
]

fig = go.Figure(data=[
    go.Pie(
        labels=channels,
        values=values,
        hole=0.4,
        marker_colors=['#636EFA', '#00CC96', '#FFA15A', '#EF553B'],
        textinfo='label+percent+value',
        texttemplate='<b>%{label}</b><br>%{value:,}<br>(%{percent})',
        hovertemplate='<b>%{label}</b><br>Transactions: %{value:,}<br>Percentage: %{percent}<extra></extra>'
    )
])

fig.update_layout(
    title=f"Real-time Channel Distribution (Last 15 min) - Total: {sum(values):,}",
    height=500,
    annotations=[
        dict(text=f'{sum(values):,}', x=0.5, y=0.5, font_size=20, showarrow=False)]
)

fig.show()

In [15]:
realtime_channel_distribution_df

,snapshot_time,current_hour,transactions_current_hour,volume_current_hour,transactions_last_15min,volume_last_15min,transactions_last_5min,transactions_per_minute_15min,transactions_per_minute_5min,fraud_alerts_15min,...,avg_transactions_per_customer,transactions_same_hour_yesterday,change_vs_yesterday_pct,system_status,mobile_15min,online_15min,atm_15min,branch_15min,mobile_pct,online_pct
0,2025-12-27 09:21:19.405327+00:00,2025-12-27 09:00:00+00:00,0,None,0,None,0,0.0,0.0,0,...,None,0,None,NORMAL,0,0,0,0,None,None


## **FRAUD MONITORING**

### Realtime Fraud Alert Feed

In [9]:
realtime_fraud_alert_feed_query = "SELECT * FROM gold.analytics_realtime_fraud_alerts ORDER BY transaction_date DESC LIMIT 50"
realtime_fraud_alert_feed_df = fetch_data(realtime_fraud_alert_feed_query)

realtime_fraud_alert_feed_df['transaction_date'] = pd.to_datetime(realtime_fraud_alert_feed_df['transaction_date'])

fig = px.scatter(
    realtime_fraud_alert_feed_df,
    x='transaction_date',
    y='fraud_score_pct',
    size='transaction_amount_abs',
    color='alert_priority',
    hover_data={
        'transaction_amount_abs': ':$,.2f',
        'merchant_category': True,
        'channel': True,
        'fraud_score_pct': ':.1f%',
        'recommended_action': True,
        'distance_from_home_km': ':.1f km',
        'velocity_24h': ':,'
    },
    title='Real-time Fraud Alerts (Last Hour)',
    labels={
        'fraud_score_pct': 'Fraud Score (%)', 'transaction_date': 'Time'},
    color_discrete_map={
        'LOW': '#00CC96',
        'MEDIUM': '#FFA15A',
        'HIGH': '#EF553B',
        'CRITICAL': '#AB63FA'
    }
)

# Add threshold line
fig.add_hline(y=80, line_dash="dash", line_color="red",
                annotation_text="Critical Threshold", annotation_position="right")

fig.update_layout(height=500)
fig.show()

In [10]:
realtime_fraud_alert_feed_df

,transaction_id,transaction_date,minutes_ago,customer_key,customer_segment,churn_risk_category,transaction_amount_abs,merchant_category,channel,is_international,...,confirmed_fraud,distance_from_home_km,merchant_risk_score,velocity_24h,amount_deviation_score,alert_priority,recommended_action,investigation_status,assigned_to,alert_generated_at


### Realtime Fraud Priority Funnel

In [11]:
realtime_fraud_priority_funnel_query = """
    SELECT alert_priority, 
        COUNT(*) as alert_count,
        SUM(transaction_amount_abs) as total_amount
    FROM gold.analytics_realtime_fraud_alerts
    GROUP BY alert_priority
    ORDER BY CASE alert_priority 
        WHEN 'CRITICAL' THEN 1 
        WHEN 'HIGH' THEN 2 
        WHEN 'MEDIUM' THEN 3 
        WHEN 'LOW' THEN 4 
        END
"""

realtime_fraud_priority_funnel_df = fetch_data(realtime_fraud_priority_funnel_query)

fig = go.Figure(go.Funnel(
    y=realtime_fraud_priority_funnel_df['alert_priority'],
    x=realtime_fraud_priority_funnel_df['alert_count'],
    textinfo="value+percent initial",
    marker={"color": ['#AB63FA', '#EF553B', '#FFA15A', '#00CC96']},
    hovertemplate='<b>%{y}</b><br>Alerts: %{x}<br>Amount: $%{customdata:,.2f}<extra></extra>',
    customdata=realtime_fraud_priority_funnel_df['total_amount']
))

fig.update_layout(
    title="Real-time Fraud Alert Priority Distribution",
    height=500
)

fig.show()


In [16]:
realtime_fraud_priority_funnel_df

,alert_priority,alert_count,total_amount


## **CUSTOMER ACTIVITY**

### Realtime Customer Activity Stream

In [13]:
realtime_customer_activity_stream_query = """
    SELECT * FROM gold.analytics_realtime_customer_activity 
    ORDER BY transactions_last_hour DESC LIMIT 20
"""
realtime_customer_activity_stream_df = fetch_data(realtime_customer_activity_stream_query)

fig = go.Figure()

fig.add_trace(go.Bar(
    x=realtime_customer_activity_stream_df['customer_natural_key'],
    y=realtime_customer_activity_stream_df['transactions_last_hour'],
    marker_color=[
        '#AB63FA' if status.startswith('ALERT: Fraud') else
        '#EF553B' if status.startswith('ALERT: Unusual') else
        '#FFA15A' if status.startswith('INFO:') else
        '#00CC96'
        for status in realtime_customer_activity_stream_df['activity_status']
    ],
    text=realtime_customer_activity_stream_df['activity_status'],
    textposition='outside',
    textangle=-45,
    hovertemplate='<b>Customer %{x}</b><br>' +
                    'Transactions: %{y}<br>' +
                    'Volume: $%{customdata[0]:,.2f}<br>' +
                    'Status: %{customdata[1]}<extra></extra>',
    customdata=realtime_customer_activity_stream_df[['volume_last_hour', 'activity_status']].values
))

fig.update_layout(
    title="Real-time Customer Activity (Top 20, Last Hour)",
    xaxis_title="Customer ID",
    yaxis_title="Transactions",
    height=600,
    showlegend=False
)
fig.show()

In [17]:
realtime_customer_activity_stream_df

,customer_key,customer_natural_key,customer_segment,churn_risk_category,transactions_last_hour,volume_last_hour,last_transaction_time,minutes_since_last_transaction,channels_used,categories_used,fraud_alerts,high_value_transactions,international_transactions,activity_level,unusual_frequency,unusual_volume,activity_status,snapshot_time


### Realtime Activity Level Pie 

In [14]:
realtime_activity_level_pie_query = """
    SELECT activity_level, COUNT(*) as customer_count
    FROM gold.analytics_realtime_customer_activity
    GROUP BY activity_level
"""
realtime_activity_level_pie_df = fetch_data(realtime_activity_level_pie_query)

color_map = {
    'Very High': '#AB63FA',
    'High': '#EF553B',
    'Moderate': '#FFA15A',
    'Low': '#00CC96'
}

fig = go.Figure(data=[
    go.Pie(
        labels=realtime_activity_level_pie_df['activity_level'],
        values=realtime_activity_level_pie_df['customer_count'],
        marker_colors=[color_map.get(level, 'gray') for level in realtime_activity_level_pie_df['activity_level']],
        textinfo='label+percent+value',
        hovertemplate='<b>%{label}</b><br>Customers: %{value:,}<br>%{percent}<extra></extra>'
    )
])

fig.update_layout(
    title="Real-time Activity Level Distribution",
    height=500
)
fig.show()

In [18]:
realtime_activity_level_pie_df

,activity_level,customer_count


## **SYSTEM HEALTH**

### Realtime System Health Indicators 

In [20]:
realtime_system_health_indicators_query = "SELECT * FROM gold.analytics_realtime_system_health"
realtime_system_health_indicators_df = fetch_data(realtime_system_health_indicators_query)

fig = make_subplots(
    rows=2, cols=3,
    specs=[[{"type": "indicator"}] * 3,
            [{"type": "indicator"}] * 3],
    subplot_titles=("Trans/Min", "Success Rate", "Processing Time",
                    "High Risk Count", "System Status", "Performance")
)

# Transactions per minute
fig.add_trace(
    go.Indicator(
        mode="number+delta",
        value=realtime_system_health_indicators_df['transactions_per_minute'].iloc[0],
        delta={'reference': 100, 'valueformat': '.1f'},
        number={'suffix': " /min"},
        title={'text': "Throughput"}
    ),
    row=1, col=1
)

# Success rate
fig.add_trace(
    go.Indicator(
        mode="gauge+number",
        value=realtime_system_health_indicators_df['success_rate_pct'].iloc[0],
        title={'text': "Success %"},
        gauge={
            'axis': {'range': [0, 100]},
            'bar': {'color': "darkgreen"},
            'steps': [
                {'range': [0, 90], 'color': "red"},
                {'range': [90, 95], 'color': "yellow"},
                {'range': [95, 100], 'color': "lightgreen"}
            ]
        }
    ),
    row=1, col=2
)

# Processing time
fig.add_trace(
    go.Indicator(
        mode="gauge+number",
        value=realtime_system_health_indicators_df['avg_processing_time_ms'].iloc[0],
        title={'text': "Proc Time (ms)"},
        gauge={
            'axis': {'range': [0, 2000]},
            'bar': {'color': "darkblue"},
            'steps': [
                {'range': [0, 500], 'color': "lightgreen"},
                {'range': [500, 1000], 'color': "yellow"},
                {'range': [1000, 2000], 'color': "red"}
            ]
        }
    ),
    row=1, col=3
)

# High risk count
fig.add_trace(
    go.Indicator(
        mode="number",
        value=realtime_system_health_indicators_df['high_risk_transactions'].iloc[0],
        title={'text': "High Risk"},
        number={'font': {
            'color': 'red' if realtime_system_health_indicators_df['high_risk_transactions'].iloc[0] > 20 else 'green'}}
    ),
    row=2, col=1
)

# System status
status_value = 1 if realtime_system_health_indicators_df['system_status'].iloc[0] == 'HEALTHY' else 0
fig.add_trace(
    go.Indicator(
        mode="number",
        value=status_value,
        title={'text': realtime_system_health_indicators_df['system_status'].iloc[0]},
        number={'font': {'color': 'green' if status_value ==
                            1 else 'red', 'size': 40}}
    ),
    row=2, col=2
)

# Performance status
perf_map = {'Excellent': 4, 'Good': 3, 'Fair': 2, 'Poor': 1}
perf_value = perf_map.get(realtime_system_health_indicators_df['performance_status'].iloc[0], 0)
fig.add_trace(
    go.Indicator(
        mode="gauge+number",
        value=perf_value,
        title={'text': realtime_system_health_indicators_df['performance_status'].iloc[0]},
        gauge={
            'axis': {'range': [0, 4], 'tickvals': [1, 2, 3, 4],
                        'ticktext': ['Poor', 'Fair', 'Good', 'Excellent']},
            'bar': {'color': ['red', 'orange', 'yellow', 'green'][perf_value-1] if perf_value > 0 else 'gray'}
        }
    ),
    row=2, col=3
)

fig.update_layout(
    title_text=f"Real-time System Health Dashboard ({realtime_system_health_indicators_df['snapshot_time'].iloc[0]})",
    height=700
)

fig.show()

### Realtime Channel health

In [21]:
realtime_channel_health_query = "SELECT * FROM gold.analytics_realtime_system_health"
realtime_channel_health_df = fetch_data(realtime_channel_health_query)

channels = ['Mobile', 'Online', 'ATM', 'Branch']
status = [realtime_channel_health_df['mobile_status'].iloc[0], realtime_channel_health_df['online_status'].iloc[0],
            realtime_channel_health_df['atm_status'].iloc[0], realtime_channel_health_df['branch_status'].iloc[0]]
transactions = [realtime_channel_health_df['mobile_transactions_5min'].iloc[0], realtime_channel_health_df['online_transactions_5min'].iloc[0],
                realtime_channel_health_df['atm_transactions_5min'].iloc[0], realtime_channel_health_df['branch_transactions_5min'].iloc[0]]

fig = go.Figure()

fig.add_trace(go.Bar(
    x=channels,
    y=transactions,
    marker_color=['green' if s == 'UP' else 'red' for s in status],
    text=[f"{s}<br>{t:,}" for s, t in zip(status, transactions)],
    textposition='outside',
    hovertemplate='<b>%{x}</b><br>Status: %{customdata[0]}<br>Transactions: %{y:,}<extra></extra>',
    customdata=[[s] for s in status]
))

fig.update_layout(
    title="Real-time Channel Health & Activity (Last 5 min)",
    xaxis_title="Channel",
    yaxis_title="Transactions",
    height=500
)

fig.show()

## **TRENDING ANALYSIS**

### Realtime Tranding Merchants

In [23]:
realtime_trending_merchants_query = """
SELECT * FROM gold.analytics_realtime_trending_merchants 
ORDER BY velocity_per_minute DESC LIMIT 20
"""
realtime_trending_merchants_df = fetch_data(realtime_trending_merchants_query)

fig = go.Figure()

fig.add_trace(go.Bar(
    x=realtime_trending_merchants_df['merchant_name'],
    y=realtime_trending_merchants_df['velocity_per_minute'],
    marker_color=realtime_trending_merchants_df['growth_rate_pct'],
    marker_colorscale='RdYlGn',
    marker_showscale=True,
    marker_colorbar=dict(title="Growth %"),
    text=realtime_trending_merchants_df['trend_indicator'],
    textposition='outside',
    hovertemplate='<b>%{x}</b><br>' +
                    'Velocity: %{y:.2f}/min<br>' +
                    'Growth: %{marker.color:.1f}%<br>' +
                    'Current Hour: %{customdata:,}<extra></extra>',
    customdata=realtime_trending_merchants_df['transactions_current_hour']
))

fig.update_layout(
    title="Real-time Trending Merchants (Velocity per Minute)",
    xaxis_title="Merchant",
    yaxis_title="Transactions per Minute",
    height=600,
    xaxis={'tickangle': -45}
)
fig.show()

In [24]:
realtime_trending_merchants_df

,merchant_name,category,merchant_type,transactions_current_hour,volume_current_hour,transactions_15min,unique_customers_current_hour,transactions_previous_hour,transaction_change,growth_rate_pct,velocity_per_minute,trend_indicator,activity_rank,snapshot_time


### Realtime Account Alerts

In [28]:
realtime_account_alerts_query = """
SELECT * FROM gold.analytics_realtime_account_alerts 
ORDER BY alert_priority, current_balance DESC
"""
realtime_account_alerts_df = fetch_data(realtime_account_alerts_query)

top_alerts_df = realtime_account_alerts_df.sort_values(
    ['alert_priority', 'current_balance'], ascending=[True, False]
)


# Conditional colors for priority
priority_colors = ['#EF553B' if p == 1 else '#FFA15A' if p == 2 else '#00CC96' for p in top_alerts_df['alert_priority']]

# Alternate row colors for readability
row_colors = []
for i in range(len(top_alerts_df)):
    base_color = '#F8F9FA' if i % 2 == 0 else 'white'
    row_colors.append([base_color]*6)

fig = go.Figure(data=[go.Table(
    header=dict(
        values=['Priority', 'Account', 'Alert Type', 'Balance', 'Action', 'Days Until Due'],
        fill_color='#636EFA',
        font=dict(color='white', size=12),
        align='left'
    ),
    cells=dict(
        values=[
            top_alerts_df['alert_priority'],
            top_alerts_df['account_natural_key'],
            top_alerts_df['alert_type'],
            ['$' + f"{b:,.2f}" for b in top_alerts_df['current_balance']],
            top_alerts_df['recommended_action'],
            top_alerts_df['days_until_due']
        ],
        fill_color=[row_colors[i] for i in range(len(row_colors))],
        font=dict(color='black', size=11),
        align='left',
        height=30
    )
)])

fig.update_layout(
    title="Real-time Account Alerts (Top 30 by Priority)",
    height=700,
    margin=dict(l=50, r=50, t=80, b=50)
)

fig.show()

In [27]:
realtime_account_alerts_df

,account_natural_key,customer_natural_key,customer_segment,product_name,account_status,current_balance,available_balance,credit_limit,credit_utilization_pct,is_past_due,is_near_limit,payment_due_date,alert_type,alert_priority,recommended_action,days_until_due,alert_time
0,35217,11685,Business,Premium Credit Card,Active,-1241.91,-1241.91,1206.70,102.92,False,True,2025-12-30,HIGH: Near Credit Limit,2,Credit Limit Review,3,2025-12-27 09:31:47.664522+00:00
1,32556,10807,Premium,Premium Credit Card,Active,-1629.66,-1629.66,1105.19,147.46,False,True,2026-01-06,HIGH: Near Credit Limit,2,Credit Limit Review,10,2025-12-27 09:31:47.664522+00:00
2,44018,14632,Affluent,Credit Card,Active,-1667.45,-1667.45,1394.98,119.53,False,True,2026-01-05,HIGH: Near Credit Limit,2,Credit Limit Review,9,2025-12-27 09:31:47.664522+00:00
3,1031,356,Premium,Premium Credit Card,Active,-1738.90,-1738.90,1215.10,143.11,False,True,2025-12-28,HIGH: Near Credit Limit,2,Credit Limit Review,1,2025-12-27 09:31:47.664522+00:00
4,59761,19873,Mass Market,Premium Credit Card,Active,-1809.39,-1809.39,1531.09,118.18,False,True,2025-12-30,HIGH: Near Credit Limit,2,Credit Limit Review,3,2025-12-27 09:31:47.664522+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,27025,8984,Premium,Premium Credit Card,Active,-4661.43,-4661.43,2377.20,196.09,False,True,2026-01-04,HIGH: Near Credit Limit,2,Credit Limit Review,8,2025-12-27 09:31:47.664522+00:00
96,64406,21433,Premium,Premium Credit Card,Active,-4668.26,-4668.26,2652.68,175.98,False,True,2026-01-24,HIGH: Near Credit Limit,2,Credit Limit Review,28,2025-12-27 09:31:47.664522+00:00
97,13920,4632,Premium,Credit Card,Active,-4679.28,-4679.28,2762.65,169.38,False,True,2026-01-15,HIGH: Near Credit Limit,2,Credit Limit Review,19,2025-12-27 09:31:47.664522+00:00
98,10878,3622,Premium,Credit Card,Active,-4686.33,-4686.33,1213.41,386.21,False,True,2026-01-02,HIGH: Near Credit Limit,2,Credit Limit Review,6,2025-12-27 09:31:47.664522+00:00
